In [1]:
%pip install -q git+https://github.com/huggingface/transformers.git

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install captum

Note: you may need to restart the kernel to use updated packages.


In [4]:
from PIL import Image
from torchvision import transforms
import pandas as pd
import os
import csv
import json

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image

import torch
import torchvision.transforms as transforms
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from transformers import AutoProcessor
from transformers import AutoModelForCausalLM


from captum.attr import TokenReferenceBase, configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [6]:
from transformers import AutoProcessor, AutoTokenizer
from transformers import AutoModelForCausalLM

output_dir = "/hdd2/datasets/ImageClef2022medCaption/clef2022/git_weights"
processor = AutoProcessor.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model.eval()

GitForCausalLM(
  (git): GitModel(
    (embeddings): GitEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(1024, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (image_encoder): GitVisionModel(
      (vision_model): GitVisionTransformer(
        (embeddings): GitVisionEmbeddings(
          (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
          (position_embedding): Embedding(197, 768)
        )
        (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (encoder): GitVisionEncoder(
          (layers): ModuleList(
            (0-11): 12 x GitVisionEncoderLayer(
              (self_attn): GitVisionAttention(
                (k_proj): Linear(in_features=768, out_features=768, bias=True)
                (v_proj): Linear(in_features=768, out_features=768, bias=True)
             

In [7]:
import os
import pandas as pd
clef2022_valid_path = '/hdd2/datasets/ImageClef2022medCaption/clef2022/Valid'
test_df = pd.read_csv(os.path.join(clef2022_valid_path, 'caption_prediction_valid.csv'),sep = '\t')
test_df['ID'] = test_df['ID'].apply(lambda x: os.path.join(clef2022_valid_path, 'valid', x + '.jpg'))
test_df.reset_index(inplace = True, drop = True)

test_df.head()
test_df_images = test_df['ID']
csv_path2 = "/hdd2/datasets/ImageClef2022medCaption/clef2022/Valid/test_results/git_pred_result_80_5.csv"
pred_df = pd.read_csv(os.path.join(csv_path2),sep = '\t')
prediction_captions = pred_df['pred']

In [8]:

# we need this wrapper
def modified_f(pixel_values):

    generated_ids = model.generate(pixel_values=pixel_values, max_length=80)
    
    return generated_ids


In [9]:
import torch
from captum.attr import IntegratedGradients
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F

# Assuming you have already loaded and preprocessed the image and caption
# input_image and target_caption are already defined

# Initialize the attribution algorithm (replace modified_f with your model)
ig = IntegratedGradients(modified_f)

img_path = test_df_images[0]
image = Image.open(img_path)
img = image.resize((224, 224), resample=Image.BILINEAR)
inputs = processor(images=img, return_tensors="pt")
pixel_values = inputs.pixel_values
zeros_tensor = torch.zeros_like(pixel_values)

generated_caption = prediction_captions[0]
# Tokenize the target caption (correction)
caption = processor(generated_caption, return_tensors='pt')

tokenized_caption = caption.input_ids

In [10]:


outputs = model(input_ids=caption.input_ids,
                attention_mask=caption.attention_mask,
                pixel_values=pixel_values,
                labels=caption.input_ids)
predicted_token_indices = outputs.logits
output_shape = outputs.logits.shape
print(output_shape)

torch.Size([1, 212, 30522])


In [11]:
outputs.logits

tensor([[[ -5.1113, -15.4328, -15.4384,  ..., -15.4288, -15.4368, -15.4246],
         [ -5.0940, -15.4607, -15.4663,  ..., -15.4567, -15.4647, -15.4525],
         [ -5.0941, -15.4607, -15.4663,  ..., -15.4567, -15.4647, -15.4525],
         ...,
         [ -3.2181, -11.6410, -11.6397,  ..., -11.6400, -11.6411, -11.6407],
         [ -7.6214, -11.0595, -11.0556,  ..., -11.0607, -11.0576, -11.0632],
         [ 25.4201, -14.0492, -14.0516,  ..., -14.0468, -14.0514, -14.0447]]],
       grad_fn=<ViewBackward0>)

In [12]:

for i in range(len(predicted_token_indices[0])):
    ppp= predicted_token_indices[0][i]
    #print(ppp)
    probabilities = F.softmax(ppp, dim=-1)
    #print(probabilities)
    next_token = torch.argmax(probabilities)  # Greedy sampling
    #print(next_token)


    # Decode the generated token IDs to get the text
    generated_text = tokenizer.decode(next_token, skip_special_tokens=True)
    print(generated_text)

o
o
o
o
o
the
the
the
the
the
the
o
o
o
o
o
o
o
o
o
o
the
the
the
o
o
o
o
o
o
o
o
o
o
o
the
o
the
o
o
o
o
o
o
o
o
o
o
o
the
o
the
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
the
o
o
o
o
o
o
o
o
o
o
o
o
o
o
o
the
right
corona
##ry
artery
be
see
in
the
right
anterior
oblique
view




In [13]:
probabilities = F.softmax(predicted_token_indices, dim=-1)
next_token = torch.argmax(probabilities)  # Greedy sampling
print(next_token)

tensor(6440142)


### attribution score

In [14]:
import torch
from PIL import Image
from captum.attr import IntegratedGradients


img_path = test_df_images[0]
image = Image.open(img_path)
img = image.resize((224, 224), resample=Image.BILINEAR)
inputs = processor(images=img, return_tensors="pt")
pixel_values = inputs.pixel_values

# Generate caption
generated_ids = modified_f(pixel_values)

# Create IntegratedGradients object
integrated_gradients = IntegratedGradients(modified_f)

# Iterate over each word in the generated caption
for i in range(1, generated_ids.shape[1]):  # Start from 1 to skip the <s> token
    target_token = generated_ids[0, i].unsqueeze(0)
    attributions, _ = integrated_gradients.attribute(
        inputs.pixel_values,
        baselines=pixel_values * 0,
        target=target_token,
        return_convergence_delta=True
    )
    
    # Normalize and visualize the attribution scores as a saliency map
    saliency_map = attributions.sum(dim=1).sum(dim=1).squeeze()
    saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min())
    
    # Overlay saliency_map on the input image
    saliency_map_image = Image.fromarray((255 * saliency_map.cpu().numpy()).astype('uint8'))
    saliency_map_image = saliency_map_image.resize(image.size)
    blended_image = Image.blend(image.convert('RGBA'), saliency_map_image.convert('RGBA'), alpha=0.5)

    blended_image.show()


IndexError: index 1996 is out of bounds for dimension 1 with size 26

In [ ]:
import torch
from captum.attr import IntegratedGradients

img_path = test_df_images[0]
image = Image.open(img_path)
img = image.resize((224, 224), resample=Image.BILINEAR)
inputs = processor(images=img, return_tensors="pt")
pixel_values = inputs.pixel_values
zeros_tensor = torch.zeros_like(pixel_values)

# Tokenize the input image
#inputs = tokenizer(image, return_tensors="pt")

# Enable gradients for the input tensor
inputs.pixel_values.requires_grad_(True)

# Compute the model's output (caption predictions)
generated_ids = modified_f(pixel_values)

print(generated_ids.shape)

# Get the size of the tensor
batch_size, seq_length = generated_ids.size()

# Create the IntegratedGradients object
integrated_gradients = IntegratedGradients(modified_f)

# Define the reference (baseline) input for the image pixels
baseline_image = torch.zeros_like(inputs.pixel_values)

# Initialize a list to store the attributions for each word in the caption
word_attributions = []

# Iterate over each position in the sequence
for i in range(seq_length):
    # Create a new tensor with zeros
    target_tensor = torch.zeros(batch_size, seq_length, dtype=torch.long)
    
    # Set the current position to the original value
    target_tensor[:, i] = generated_ids[:, i]
    #print(len(target_tensor[0]))
    
    # Compute the integrated gradients for the input image for the i-th word
    attributions, _ = integrated_gradients.attribute(inputs.pixel_values, baseline_image, target=target_tensor, return_convergence_delta=True)

    # Sum the attributions along the channel dimension to get a single score for each word
    word_attribution_score = attributions.sum(dim=1).sum(dim=1)
    word_attributions.append(word_attribution_score)

# Reset gradients for the input tensor
inputs.pixel_values.requires_grad_(False)

# The list `word_attributions` now contains the attribution scores for each word in the caption.
# You can use this information to interpret and analyze how each word contributes to the model's prediction.



In [ ]:
# Compute the attributions for each word in the caption
for i in range(len(generated_ids[0])):
    # Set the label (output target) to be the i-th word in the caption
    target_label = generated_ids[:, i].unsqueeze(1)
    
    # Compute the integrated gradients for the input image for the i-th word
    attributions, _ = integrated_gradients.attribute(inputs.pixel_values, baseline_image, target=target_label, return_convergence_delta=True)

    # Sum the attributions along the channel dimension to get a single score for each word
    word_attribution_score = attributions.sum(dim=1).sum(dim=1)
    word_attributions.append(word_attribution_score)

# Reset gradients for the input tensor
inputs.pixel_values.requires_grad_(False)

# The list `word_attributions` now contains the attribution scores for each word in the caption.
# You can use this information to interpret and analyze how each word contributes to the model's prediction.

### visualization

In [ ]:
# vec_ten = []
# for i in range(len(tokenized_caption[0])):
#     vec = torch.zeros_like(tokenized_caption[0])  # Create a new tensor with all elements as 0
#     vec[i] = tokenized_caption[0][i]              # Set one element to the value in the original tensor
#     vec_ten.append(vec)

# print(vec_ten)

In [ ]:
# for i, word_id in enumerate(vec_ten):
#     print(word_id)

In [ ]:

# # Create a list to store the attribution scores for each word
# word_attributions = []

# import torch
# from PIL import Image

# # Assuming 'processor' and 'ig' are initialized, and you have 'pixel_values' and 'zeros_tensor' as mentioned earlier

# # Assuming 'tokenized_caption' contains the token IDs of the caption
# # Iterate through each word ID in the tokenized caption
# for i, word_id in enumerate(vec_ten):
#     # Compute the attribution scores for the current word ID
#     attributions, _ = ig.attribute(inputs=pixel_values, baselines=zeros_tensor, target=word_id, return_convergence_delta=True)

#     # Assuming attributions is a tensor, you can aggregate the scores to get a single value
#     word_attribution_score = torch.sum(attributions).item()

#     # Store the attribution score for the current word ID
#     word_attributions.append((processor.decode(word_id), word_attribution_score))


# # Visualize the attribution scores as a heatmap on the image
# attributions_heatmap = attributions.sum(dim=0)  # Aggregate word-level attributions
# attributions_heatmap = attributions_heatmap.cpu().detach().numpy()
# attributions_heatmap = attributions_heatmap / attributions_heatmap.max()  # Normalize scores to [0, 1]

# plt.imshow(img)
# plt.imshow(attributions_heatmap, alpha=0.7, cmap='jet', interpolation='bilinear')
# plt.axis('off')
# plt.show()

In [ ]:
import torch

# Your original tensor
original_tensor = torch.tensor([[101, 1996, 2157, 21887, 2854, 16749, 2022, 2156, 1999, 1996, 2157, 15099, 20658, 3193, 102]])

# Get the size of the tensor
batch_size, seq_length = original_tensor.size()

# Iterate over each position in the sequence
for i in range(seq_length):
    # Create a new tensor with zeros
    new_tensor = torch.zeros(batch_size, seq_length, dtype=torch.long)
    
    # Set the current position to the original value
    new_tensor[:, i] = original_tensor[:, i]
    
    # Print the new tensor
    print(new_tensor)
